In [ ]:
import re
import sys

import numpy as np
import matplotlib.pyplot as plt

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

In [ ]:
data_paths = pymedphys.zip_data_paths("monaco_trf_compare.zip")
static_angles = [path for path in data_paths if path.parent.name == "StaticAnglesNoCol"]

In [ ]:
def get_file_type(input_paths, file_type):
    paths = [path for path in input_paths if file_type in path.name]
    assert len(paths) == 1
    return paths[0]
    
    
tel_path = get_file_type(static_angles, "tel")
trf_path = get_file_type(static_angles, "trf")

In [ ]:
delivery_tel = pymedphys.Delivery.from_monaco(tel_path)
delivery_trf = pymedphys.Delivery.from_logfile(trf_path)

In [ ]:
mu_density_angles = set(delivery_tel.gantry)

In [ ]:
mudensity_tel = delivery_tel.mudensity(gantry_angles=mu_density_angles)

In [ ]:
mudensity_trf = delivery_trf.mudensity(gantry_angles=mu_density_angles)

In [ ]:
coords = (grid['jaw'], grid['mlc'])

In [ ]:
gammas = []
percent_deviation = 1
mm_dist_threshold = 0.2

for tel_result, trf_result in zip(mudensity_tel, mudensity_trf):
    gammas.append(
        pymedphys.gamma(coords, tel_result, coords, trf_result, percent_deviation, mm_dist_threshold, local_gamma=True))

In [ ]:
def plot_gamma_hist(gamma, percent, dist):
    valid_gamma = gamma[~np.isnan(gamma)]

    plt.hist(valid_gamma, 50, density=True)
    pass_ratio = np.sum(valid_gamma <= 1) / len(valid_gamma)

    plt.title("Local Gamma ({0}%/{1}mm) | Percent Pass: {2:.2f} % | Max Gamma: {3:.2f}".format(percent, dist, pass_ratio*100, np.max(valid_gamma)))

In [ ]:
grid = pymedphys.mudensity.grid()

In [ ]:
for gantry_angle, tel_result, trf_result, gamma in zip(mu_density_angles, mudensity_tel, mudensity_trf, gammas):
    print(f"Gantry Angle = {gantry_angle}")
    
    plt.figure()
    pymedphys.mudensity.display(grid, tel_result)
    plt.title("Monaco Plan")
    
    plt.figure()
    pymedphys.mudensity.display(grid, tel_result)
    plt.title("Logfile Result")
    
    diff = trf_result - tel_result
    largest_item = np.max(np.abs(diff))
    
    plt.figure()
    pymedphys.mudensity.display(grid, diff, cmap="seismic", vmin=-largest_item, vmax=largest_item)
    plt.title("Logfile - Monaco")
    
    plt.figure()
    pymedphys.mudensity.display(grid, gamma, cmap="coolwarm", vmin=0, vmax=2)
    plt.title(f"Local Gamma | {percent_deviation}%/{mm_dist_threshold}mm")
    
    plt.figure()
    plot_gamma_hist(gamma, percent_deviation, mm_dist_threshold)
    
    plt.show()